In [1]:
# Import needed dependencies
import requests
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import re
import time
from datetime import date
from bs4 import BeautifulSoup, Comment

In [2]:
team_codes_df = pd.read_csv('team_codes.csv')
games_attended_df = pd.read_csv('sports_attendance.csv')

games_attended_df["URL_Date"] = " "
games_attended_df["Year"] = " "

games_attended_df['Year'] = pd.to_datetime(games_attended_df.Date)
games_attended_df['Year'] = games_attended_df['Year'].dt.strftime('%Y')
games_attended_df['URL_Date'] = pd.to_datetime(games_attended_df.Date)
games_attended_df['URL_Date'] = games_attended_df['URL_Date'].dt.strftime('%Y%m%d0')

nba_attendance_df = games_attended_df[games_attended_df.Sport == ('NBA')].copy()
mlb_attendance_df = games_attended_df[games_attended_df.Sport == ('MLB')].copy()
nfl_attendance_df = games_attended_df[games_attended_df.Sport == ('NFL')].copy()

mlb_df = mlb_attendance_df.merge(team_codes_df, how='left', left_on='Home', right_on='MLB_Teams')
mlb_df = mlb_df.drop(columns=['NBA_Teams', 'NBA_Codes', 'NBA_Teams', 'NFL_Teams', 'NFL_Codes', 'NHL_Teams', 'NHL_Codes', 'MLS_Codes'])

mlb_df['URL_Variable'] = mlb_df['MLB_Codes'] + mlb_df['URL_Date']

away_code_df = mlb_attendance_df.merge(team_codes_df, how='left', left_on='Visitor', right_on='MLB_Teams')
away_code_df = away_code_df.drop(columns=['NBA_Teams', 'NBA_Codes', 'NBA_Teams', 'NFL_Teams', 'NFL_Codes', 'NHL_Teams', 'NHL_Codes', 'MLS_Codes'])

mlb_df['Away_Code'] = away_code_df['MLB_Codes']
mlb_df.rename(columns = {'MLB_Codes':'Home_Code'}, inplace = True)
mlb_df


,Date,Home,Visitor,Sport,URL_Date,Year,MLB_Teams,Home_Code,URL_Variable,Away_Code
0,7/27/2007,San Francisco Giants,Florida Marlins,MLB,200707270,2007,San Francisco Giants,SFN,SFN200707270,FLO
1,7/28/2007,San Francisco Giants,Florida Marlins,MLB,200707280,2007,San Francisco Giants,SFN,SFN200707280,FLO
2,9/5/2008,St. Louis Cardinals,Florida Marlins,MLB,200809050,2008,St. Louis Cardinals,SLN,SLN200809050,FLO
3,9/6/2008,St. Louis Cardinals,Florida Marlins,MLB,200809060,2008,St. Louis Cardinals,SLN,SLN200809060,FLO
4,7/25/2009,Los Angeles Angels of Anaheim,Minnesota Twins,MLB,200907250,2009,Los Angeles Angels of Anaheim,ANA,ANA200907250,MIN
...,...,...,...,...,...,...,...,...,...,...
62,4/6/2024,St. Louis Cardinals,Miami Marlins,MLB,202404060,2024,St. Louis Cardinals,SLN,SLN202404060,MIA
63,4/7/2024,St. Louis Cardinals,Miami Marlins,MLB,202404070,2024,St. Louis Cardinals,SLN,SLN202404070,MIA
64,5/10/2024,Baltimore Orioles,Arizona Diamondbacks,MLB,202405100,2024,Baltimore Orioles,BAL,BAL202405100,ARI
65,5/11/2024,New York Mets,Atlanta Braves,MLB,202405110,2024,New York Mets,NYN,NYN202405110,ATL


In [3]:
URL_list = mlb_df["URL_Variable"].values.tolist()
Home_code_list = mlb_df["Home_Code"].values.tolist()
Away_code_list = mlb_df["Away_Code"].values.tolist()
Home_team_list = mlb_df["Home"].values.tolist()
Away_team_list = mlb_df["Visitor"].values.tolist()
Year_list = mlb_df["Year"].values.tolist()
Date_list = mlb_df["URL_Date"].values.tolist()
Actual_date_list = mlb_df["Date"].values.tolist()

In [4]:
coded_home_teams = []
coded_away_teams = []
pitching_bbreference_code_list = []

for item in Home_team_list: 
    home_code = re.sub(r'[^a-zA-Z0-9]', '', item)
    coded_home_teams.append(home_code)

for item in Away_team_list: 
    away_code = re.sub(r'[^a-zA-Z0-9]', '', item)
    coded_away_teams.append(away_code)

# Maybe someday I'll find a better way to do this, but for now Baseball Reference pitching stats are buring
## under some seemingly meaningless code, that is the same for years prior to 2010 (at least in the years I 
## went to games) but changes each year afterward. It seems to be the same for any playoff game, though



for item in Year_list:
    item = int(item)
    if item < 2010:
        pitching_bbreference_code_list.append('#all_1500181613')
    elif item == 2010:
        pitching_bbreference_code_list.append('#all_95821928')
    elif item == 2011:
        pitching_bbreference_code_list.append('#all_95821928')
    elif item == 2012:
        pitching_bbreference_code_list.append('#all_95821928')
    elif item == 2013:
        pitching_bbreference_code_list.append('#all_95821928')
    elif item == 2014:
        pitching_bbreference_code_list.append('#all_95821928')
    elif item == 2015:
        pitching_bbreference_code_list.append('#all_95821928')
    elif item == 2016:
        pitching_bbreference_code_list.append('#all_95821928')
    elif item == 2017:
        pitching_bbreference_code_list.append('#all_95821928')
    elif item == 2018:
        pitching_bbreference_code_list.append('#all_95821928')
    elif item == 2019:
        pitching_bbreference_code_list.append('#all_95821928')
    elif item == 2020:
        pitching_bbreference_code_list.append('#all_1498343846')
    elif item == 2021:
        pitching_bbreference_code_list.append('#all_8112864405')
    elif item == 2022:
        pitching_bbreference_code_list.append('#all_8169210527')
    elif item == 2023:
        pitching_bbreference_code_list.append('#all_4128382890')
    elif item == 2024:
        pitching_bbreference_code_list.append('#all_5297727181')
        
    
#  LosAngelesAngelsofAnaheim   
# pitching_bbreference_code_list

In [5]:
# getting length of list
length = len(URL_list)

dataframe_dictionary = {}

box_score_df_list = []
home_stats_df_list = []
home_pitching_stats_df_list = []
away_stats_df_list = []
away_pitching_stats_df_list = []
game_logistics_df_list = []
# pitching_line = []

for i in range(length):
    time.sleep(2)
    
    # Create empty lists to hold table data to be scraped
    away_box_score = []
    home_box_score = []
    away_pitching_box_score = []
    home_pitching_box_score = []
    box_score = []

    # input URL and use BeautifulSoup to parse through the page
    url = f'https://www.baseball-reference.com/boxes/{Home_code_list[i]}/{URL_list[i]}.shtml'
    soup = BeautifulSoup(requests.get(url).content, 'html.parser')

    # Grab the table element that has away team statistics
    away_table_placeholder = soup.select_one(f'#all_{coded_away_teams[i]}batting') 
    
### ONLY CODE FOUND TO GET PITCHING DATA GETS BOTH TEAMS AT ONCE AND HAS RANDOM NUMBERS ASSOCIATED WITH IT
### WILL NEED TO FIGURE OUT WHAT THE NUMBERS MEAN AND ADJUST CODE TO SCAPE BOTH AT ONCE.
### EXAMPLE FOR FIRST GAME    soup.select_one('#all_2420024094')
    
    # Game box score is commented out in html, so this will grab it out of the comments
    for comment in away_table_placeholder.find_all(text=lambda text: isinstance(text, Comment)):
        if comment.find("<table ") > 0:
            comment_soup = BeautifulSoup(comment, 'html.parser')
            away_table = comment_soup.find("table")

    # Grab data from table and put it into the list created above
    for tr in away_table.select('tr:has(td)'):
        tds = [td.get_text(strip=True) for td in tr.select('td')]
        away_box_score.append(tds)

    # Grab the table element that has home team statistics
    home_table_placeholder = soup.select_one(f'#all_{coded_home_teams[i]}batting') 

    # Game box score is commented out in html, so this will grab it out of the comments
    for comment in home_table_placeholder.find_all(text=lambda text: isinstance(text, Comment)):
        if comment.find("<table ") > 0:
            comment_soup = BeautifulSoup(comment, 'html.parser')
            home_table = comment_soup.find("table")

    # Grab data from table and put it into the list created above
    for tr in home_table.select('tr:has(td)'):
        tds2 = [td.get_text(strip=True) for td in tr.select('td')]
        home_box_score.append(tds2)

    # Grab the table element that has game box score
    box_score_table = soup.select('[class*="linescore_wrap"]')

    # Grab data from table and put it into the list created above
    for tr in box_score_table[0].select('tr:has(td)'):
        tds3 = [td.get_text(strip=True) for td in tr.select('td')]
        box_score.append(tds3)

       
    # Create dataframe for away batting statistics
    dataframe_dictionary[Away_code_list[i] + Date_list[i]] = pd.DataFrame(away_box_score)

    # Create an empty list to store away team statistics header information
    away_header_list = []

    # Grab the table header information to use as column headers in our away batting statistics dataframe
    for tr in away_table.select('tr:has(th)'):
        ths = [th.get_text(strip=True) for th in tr.select('th')]
        away_header_list.append(ths)

    away_header_list[0].remove("Batting")    
        
    # Create dataframe for home batting statistics
    dataframe_dictionary[Home_code_list[i] + Date_list[i]] = pd.DataFrame(home_box_score)

    # Create an empty list to store home team statistics header information
    home_header_list = []
    home_pitching_header_list = []

    # Grab the table header information to use as column headers in our home team statistics dataframe
    for tr in home_table.select('tr:has(th)'):
        ths2 = [th.get_text(strip=True) for th in tr.select('th')]
        home_header_list.append(ths2)    
    
    home_header_list[0].remove("Batting")

    
    # Gather all pitching stats from each game attended
    
    try:
        pitching_table_placeholder = soup.select(pitching_bbreference_code_list[i])[0]
    except IndexError:
        # This is the playoff game pitching code
        pitching_table_placeholder = soup.select('#all_4804618167')[0]
    
    away_pitching_box_score = []
    home_pitching_box_score = []
    away_pitching_header_list = []
    home_pitching_header_list = []

    # Game pitching box score is commented out in html, so this will grab it out of the comments
    for comment in pitching_table_placeholder.find_all(text=lambda text: isinstance(text, Comment)):
        if comment.find("<table ") > 0:
            comment_soup = BeautifulSoup(comment, 'html.parser')
            pitching_table = comment_soup.find_all("table")

    # Grab away pitching data from table and put it into the list created above
    for tr in pitching_table[0].select('tr:has(td)'):
        tdsAP = [td.get_text(strip=True) for td in tr.select('td')]
        away_pitching_box_score.append(tdsAP)

    # Grab home pitching data from table and put it into the list created above
    for tr in pitching_table[1].select('tr:has(td)'):
        tdsHP = [td.get_text(strip=True) for td in tr.select('td')]
        home_pitching_box_score.append(tdsHP)

    dataframe_dictionary['P_' + Away_code_list[i] + Date_list[i]] = pd.DataFrame(away_pitching_box_score)
    dataframe_dictionary['P_' + Home_code_list[i] + Date_list[i]] = pd.DataFrame(home_pitching_box_score)

    # Grab the table header information to use as column headers in our away pitching statistics dataframe
    for tr in pitching_table[0].select('tr:has(th)'):
        thsAP = [th.get_text(strip=True) for th in tr.select('th')]
        away_pitching_header_list.append(thsAP)

    # Grab the table header information to use as column headers in our home pitching statistics dataframe
    for tr in pitching_table[1].select('tr:has(th)'):
        thsHP = [th.get_text(strip=True) for th in tr.select('th')]
        home_pitching_header_list.append(thsHP)

    away_pitching_header_list[0].remove("Pitching")   
    home_pitching_header_list[0].remove("Pitching")   
    away_pitchers = away_pitching_header_list[1:-1]
    home_pitchers = home_pitching_header_list[1:-1]

    away_pitchers_list = [item for sublist in away_pitchers for item in sublist]
    away_pitchers_list.append(f'{Away_team_list[i]} Totals')
    home_pitchers_list = [item for sublist in home_pitchers for item in sublist]
    home_pitchers_list.append(f'{Home_team_list[i]} Totals')
    dataframe_dictionary['P_' + Away_code_list[i] + Date_list[i]].index = away_pitchers_list
    dataframe_dictionary['P_' + Away_code_list[i] + Date_list[i]]['Team'] = Away_team_list[i]
    dataframe_dictionary['P_' + Away_code_list[i] + Date_list[i]]['Date'] = Actual_date_list[i]
    dataframe_dictionary['P_' + Home_code_list[i] + Date_list[i]].index = home_pitchers_list
    dataframe_dictionary['P_' + Home_code_list[i] + Date_list[i]]['Team'] = Home_team_list[i]
    dataframe_dictionary['P_' + Home_code_list[i] + Date_list[i]]['Date'] = Actual_date_list[i]


    away_pitcher_column_headers = away_pitching_header_list[0]
    away_pitcher_column_headers.append('Team')
    away_pitcher_column_headers.append('Date')
    home_pitcher_column_headers = home_pitching_header_list[0]
    home_pitcher_column_headers.append('Team')
    home_pitcher_column_headers.append('Date')

    dataframe_dictionary['P_' + Away_code_list[i] + Date_list[i]].columns = away_pitcher_column_headers
    dataframe_dictionary['P_' + Home_code_list[i] + Date_list[i]].columns = home_pitcher_column_headers
    
    
    # Create dataframe for game box score
    dataframe_dictionary['Box' + Date_list[i]] = pd.DataFrame(box_score)

    # Create an empty list to store game box score dataframe header information
    box_score_header_list = []

    # Grab the table header information to use as column headers in our game box score dataframe
    for tr in box_score_table[0].select('tr:has(th)'):
        ths3 = [th.get_text(strip=True) for th in tr.select('th')]
        box_score_header_list.append(ths3)

    box_score_header_list[0][1] = 'Team'
    dataframe_dictionary['Box' + Date_list[i]].columns = box_score_header_list

    # box_score_df.columns = box_score_list
#     final_box_score_df = box_score_df.iloc[: , 1:]
#     final_box_score_df.drop(final_box_score_df.tail(1).index,inplace=True) 

#     home_away = ['Away','Home']
#     final_box_score_df.rename(index={0:'Away'},inplace=True)
#     final_box_score_df.rename(index={1:'Home'},inplace=True)


    # If a game goes into extra innings, we need to add to the box score list for each overtime period
    Extra_inning_counter = 10
    while len(box_score[0]) > len(box_score_header_list[0]):
        box_score_header_list.insert(-1, f'{Extra_inning_counter}')
        Extra_inning_counter = Extra_inning_counter + 1
        
    # Update the column and row labels
    dataframe_dictionary['Box' + Date_list[i]] = dataframe_dictionary['Box' + Date_list[i]].set_axis(box_score_header_list, axis=1, inplace=False)
#     dataframe_dictionary['Box' + Date_list[i]] = dataframe_dictionary['Box' + Date_list[i]].set_axis([f'{Away_team_list[i]}', f'{Home_team_list[i]}'])
    
    dataframe_dictionary['Box' + Date_list[i]]['Date'] = Actual_date_list[i]
    box_score_df_list.append(dataframe_dictionary['Box' + Date_list[i]])
    
    # Pull the away team player data and combine the lists into one list
    away_starters = away_header_list[1:-1]
#     away_bench = away_header_list[8:-1]
#     away_players = [away_starters,away_bench]

    # Pull the home team player data and combine the lists into one list
    home_starters = home_header_list[1:-1]
#     home_bench = home_header_list[8:-1]
#     home_players = [home_starters,home_bench]

    # Consolodate the away team list of lists into one list
    away_starters_list = [item for sublist in away_starters for item in sublist]
#     away_bench_list = [item for sublist in away_bench for item in sublist]
#     away_roster = away_starters_list + away_bench_list

    # Consolodate the home team list of lists into one list
    home_starters_list = [item for sublist in home_starters for item in sublist]
#     home_bench_list = [item for sublist in home_bench for item in sublist]
#     home_roster = home_starters_list + home_bench_list

    # Add Team Totals to the end of the away team list
    away_starters.append('Team Totals')
    dataframe_dictionary[Away_code_list[i] + Date_list[i]].index = away_starters

    # Add Team Totals to the end of the home team list
    home_starters.append('Team Totals')
    dataframe_dictionary[Home_code_list[i] + Date_list[i]].index = home_starters

    # Set the home and away dataframe column labels
    column_headers = away_header_list[0]
#     column_headers.remove("Starters")
    dataframe_dictionary[Away_code_list[i] + Date_list[i]].columns = column_headers
    dataframe_dictionary[Home_code_list[i] + Date_list[i]].columns = column_headers

    dataframe_dictionary[Away_code_list[i] + Date_list[i]]['Date'] = Actual_date_list[i]
    dataframe_dictionary[Home_code_list[i] + Date_list[i]]['Date'] = Actual_date_list[i]
    
    dataframe_dictionary[Away_code_list[i] + Date_list[i]]['Team'] = Away_team_list[i]
    dataframe_dictionary[Home_code_list[i] + Date_list[i]]['Team'] = Home_team_list[i]
    
    home_stats_df_list.append(dataframe_dictionary[Home_code_list[i] + Date_list[i]])
    home_pitching_stats_df_list.append(dataframe_dictionary['P_' + Home_code_list[i] + Date_list[i]])
    away_stats_df_list.append(dataframe_dictionary[Away_code_list[i] + Date_list[i]])
    away_pitching_stats_df_list.append(dataframe_dictionary['P_' + Away_code_list[i] + Date_list[i]])
        
    # Pull logistical information about the game: attendance, length of game, and venue
    text = soup.get_text()
    attendance_string = re.findall(r"Attendance:\s+\d*,\d*", text)
    
    # A failsafe, as one of the games has no attendance listed causing the loop to break
    if(len(attendance_string) != 0):
        
        # Clean up the attendance string that is pulled 
        new_attendance_string = attendance_string[0].strip('([^a-z]xa)')
        attendance_list = list(new_attendance_string)
        attendance_list.pop(11)
    else:
        attendance_string = "No Attendance Available"
    
    # Clean up the game length string that is pulled 
    game_time_string = re.findall(r"Game Duration:\s+\d:\d*", text)    
    new_game_time_string = game_time_string[0].strip('([^a-z]xa)')
    game_time_list = list(new_game_time_string)
    game_time_list.pop(13)

    # join the lists to create final string
    final_attendance_string = "".join(attendance_list)
    final_game_time_sting = "".join(game_time_list)

    # Create throwaway variables to hold split strings so we can pick out what we want later
    x = final_attendance_string.split(":")
    y = final_game_time_sting.split(" ")
    y[0:2] = [' '.join(y[0:2])]

    # Pick out the strings before the : that we split above
    dataframe_dictionary['Attendance' + Date_list[i]] = pd.DataFrame(x, columns = [x[0]])
    dataframe_dictionary['Attendance' + Date_list[i]] = dataframe_dictionary['Attendance' + Date_list[i]].drop([0])
    dataframe_dictionary['Game Duration' + Date_list[i]] = pd.DataFrame(y, columns = [y[0]])
    dataframe_dictionary['Game Duration' + Date_list[i]] = dataframe_dictionary['Game Duration' + Date_list[i]].drop([0])

    # Join the attendance and game time dataframes into one dataframe
    dataframe_dictionary['Game_Logistics' + Date_list[i]] = pd.concat([dataframe_dictionary['Attendance' + Date_list[i]], dataframe_dictionary['Game Duration' + Date_list[i]]], ignore_index=True, sort=False)
    dataframe_dictionary['Game_Logistics' + Date_list[i]]['Game Duration'] = dataframe_dictionary['Game_Logistics' + Date_list[i]]['Game Duration'].shift(-1)
    dataframe_dictionary['Game_Logistics' + Date_list[i]] = dataframe_dictionary['Game_Logistics' + Date_list[i]].dropna()

    # Scrape the name of the vanue and location and add that to the logistics dataframe
    divparent = soup.find('div', attrs={'class':'scorebox_meta'})
    text = divparent.text
    result = re.split(r'\n', text)
    almost_done = result[1]
    final_result = almost_done.partition("Venue: ")[2]
    real_final_result = final_result.split("Game Duration:")[0]
    dataframe_dictionary['Game_Logistics' + Date_list[i]] = dataframe_dictionary['Game_Logistics' + Date_list[i]].assign(Venue=[real_final_result])

    dataframe_dictionary['Game_Logistics' + Date_list[i]]['Date'] = Actual_date_list[i]
    game_logistics_df_list.append(dataframe_dictionary['Game_Logistics' + Date_list[i]])
    

In [6]:
########## GUIDE TO ACCESSING DATAFRAMES ##########

## Pull Home Stats --> dataframe_dictionary['(Home Code)(Date)']

## Pull Home Pitching Stats --> dataframe_dictionary['P_(Home Code)(Date)']

## Pull Away Stats --> dataframe_dictionary['(Away Code)(Date)']

## Pull Away Pitching Stats --> dataframe_dictionary['P_(Away Code)(Date)']

## Pull Box Score --> dataframe_dictionary['Box(Date)']

## Pull Game Logistics --> dataframe_dictionary['Game_Logistics(Date)']

In [7]:
#Test
dataframe_dictionary['Game_Logistics202205070']

,Attendance,Game Duration,Venue,Date
0,"40,113",3:22,Oracle Park,5/7/2022


In [8]:
#Test
dataframe_dictionary['SFN202205070']

,AB,R,H,RBI,BB,SO,PA,BA,OBP,SLG,...,WPA+,WPA-,cWPA,acLI,RE24,PO,A,Details,Date,Team
[Austin SlaterCF],4,1,1,0,1,2,5,.255,.352,.404,...,0.048,-0.042,0.00%,0.56,-0.2,2,0,,5/7/2022,San Francisco Giants
[Mauricio DubónSS],4,4,2,2,1,0,5,.182,.200,.303,...,0.219,-0.011,0.12%,0.60,2.1,1,3,HR,5/7/2022,San Francisco Giants
[Brandon Belt1B],1,2,0,0,3,0,4,.239,.386,.463,...,0.102,-0.002,0.06%,0.66,1.3,6,0,,5/7/2022,San Francisco Giants
[LaMonte Wade Jr.PH-1B],0,1,0,0,1,0,1,.000,.333,.000,...,0.006,0.000,0.00%,0.16,0.6,1,0,,5/7/2022,San Francisco Giants
[Darin RufDH],4,2,1,2,1,1,5,.190,.296,.240,...,0.113,-0.073,0.02%,0.72,2.4,,,HR,5/7/2022,San Francisco Giants
[Wilmer Flores3B],3,1,1,6,1,0,5,.269,.340,.419,...,0.264,-0.014,0.14%,0.80,3.2,0,2,"HR,SF",5/7/2022,San Francisco Giants
[Luis GonzálezLF],4,1,1,1,0,1,5,.318,.365,.432,...,0.013,-0.019,-0.00%,0.21,0.6,2,0,HBP,5/7/2022,San Francisco Giants
[Thairo Estrada2B],4,0,1,2,0,0,5,.240,.278,.360,...,0.064,-0.008,0.03%,0.20,0.7,4,2,"2B,SF",5/7/2022,San Francisco Giants
[Joey BartC],4,0,0,0,0,2,5,.164,.313,.345,...,0.003,-0.019,-0.01%,0.18,-1.5,10,0,"HBP,GDP",5/7/2022,San Francisco Giants
[Mike YastrzemskiRF],4,1,2,0,0,0,4,.288,.354,.356,...,0.033,-0.012,0.01%,0.31,-0.1,1,0,,5/7/2022,San Francisco Giants


In [9]:
#Test
dataframe_dictionary['P_SFN202205070']

,IP,H,R,ER,BB,SO,HR,ERA,BF,Pit,...,GSc,IR,IS,WPA,aLI,cWPA,acLI,RE24,Team,Date
"Logan Webb, W (4-1)",5,8,4,4,2,7,0,3.82,24,84,...,40,,,-0.148,0.88,-0.08%,0.83,-1.6,San Francisco Giants,5/7/2022
Dominic Leone,1,1,0,0,0,2,0,2.25,4,16,...,,0,0,0.019,0.48,0.01%,0.46,0.5,San Francisco Giants,5/7/2022
Zack Littell,1,4,3,3,0,1,1,4.00,7,28,...,,0,0,-0.079,0.65,-0.04%,0.62,-2.5,San Francisco Giants,5/7/2022
"Tyler Rogers, H (7)",1,1,0,0,0,0,0,2.31,4,12,...,,0,0,0.041,0.58,0.02%,0.55,0.5,San Francisco Giants,5/7/2022
Jarlín García,1,0,0,0,1,0,0,0.00,4,19,...,,0,0,0.002,0.07,0.00%,0.07,0.5,San Francisco Giants,5/7/2022
San Francisco Giants Totals,9,14,7,7,3,10,1,7.00,43,159,...,40,0,0,-0.165,0.70,-0.09%,0.67,-2.7,San Francisco Giants,5/7/2022


In [10]:
#Test
dataframe_dictionary['Box202205070']

,,Team,1,2,3,4,5,6,7,8,9,R,H,E,Date
0,via Sports Logos.netAbout logos,St. Louis Cardinals,1,2,0,1,0,0,3,0,0,7,14,3,5/7/2022
1,via Sports Logos.netAbout logos,San Francisco Giants,4,4,0,1,0,0,1,3,X,13,9,0,5/7/2022
2,WP: Logan Webb (4-1) • LP: Steven Matz (3-2),None,None,None,None,None,None,None,None,None,None,None,None,None,5/7/2022


In [11]:
total_box_score_df = pd.concat(box_score_df_list)

total_box_score_df

,,1,10,11,12,13,14,15,2,3,...,5,6,7,8,9,Date,E,H,R,Team
0,via Sports Logos.netAbout logos,1,NaN,NaN,NaN,NaN,NaN,NaN,0,4,...,0,1,1,0,2,7/27/2007,2,15,10,Florida Marlins
1,via Sports Logos.netAbout logos,2,NaN,NaN,NaN,NaN,NaN,NaN,0,0,...,4,6,0,0,X,7/27/2007,0,14,12,San Francisco Giants
2,WP: Vinnie Chulk (4-2) • LP: Renyel Pinto (2-4),None,NaN,NaN,NaN,NaN,NaN,NaN,None,None,...,None,None,None,None,None,7/27/2007,None,None,None,None
0,via Sports Logos.netAbout logos,0,NaN,NaN,NaN,NaN,NaN,NaN,0,3,...,0,0,0,0,0,7/28/2007,2,6,3,Florida Marlins
1,via Sports Logos.netAbout logos,1,NaN,NaN,NaN,NaN,NaN,NaN,0,0,...,0,1,0,0,2,7/28/2007,0,9,4,San Francisco Giants
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,via Sports Logos.netAbout logos,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,...,0,0,0,0,1,5/11/2024,1,2,1,New York Mets
2,WP: Max Fried (3-1) • LP: Christian Scott (0-1),None,NaN,NaN,NaN,NaN,NaN,NaN,None,None,...,None,None,None,None,None,5/11/2024,None,None,None,None
0,via Sports Logos.netAbout logos,3,NaN,NaN,NaN,NaN,NaN,NaN,0,0,...,0,0,0,1,0,5/13/2024,0,10,5,Tampa Bay Rays
1,via Sports Logos.netAbout logos,3,NaN,NaN,NaN,NaN,NaN,NaN,0,0,...,0,0,0,0,0,5/13/2024,0,8,3,Boston Red Sox


In [12]:
total_home_stats_df = pd.concat(home_stats_df_list)

team_col = total_home_stats_df.pop("Team")
total_home_stats_df.insert(0, "Team", team_col)

total_home_stats_df

,Team,AB,R,H,RBI,BB,SO,PA,BA,OBP,...,aLI,WPA+,WPA-,cWPA,acLI,RE24,PO,A,Details,Date
[Dave RobertsCF],San Francisco Giants,4,1,2,0,1,1,5,.262,.327,...,1.00,0.097,-0.095,0%,0.05,0.2,3,0,SB,7/27/2007
[Omar VizquelSS],San Francisco Giants,4,1,1,1,0,0,5,.246,.295,...,1.36,0.172,-0.104,0.00%,0.06,-0.4,2,2,SH,7/27/2007
[Ray Durham2B],San Francisco Giants,4,1,2,3,0,1,5,.240,.307,...,0.96,0.272,-0.072,0.01%,0.04,1.2,1,1,"HR,2B,SF",7/27/2007
[Randy MessengerP],San Francisco Giants,0,0,0,0,0,0,0,.000,.000,...,,,,,,,0,0,,7/27/2007
[Barry BondsLF],San Francisco Giants,1,2,1,1,4,0,5,.281,.499,...,0.36,0.154,0.000,0.00%,0.02,2.1,2,0,HR,7/27/2007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
[Ceddanne RafaelaCF],Boston Red Sox,3,0,0,0,0,0,3,.209,.236,...,1.71,0.000,-0.140,-0.08%,1.60,-0.9,4,0,,5/13/2024
[],Boston Red Sox,,,,,,,,,,...,,,,,,,,,,5/13/2024
[Kutter CrawfordP],Boston Red Sox,,,,,,,,,,...,,,,,,,1,0,,5/13/2024
[Greg WeissertP],Boston Red Sox,,,,,,,,,,...,,,,,,,0,0,,5/13/2024


In [13]:
total_away_stats_df = pd.concat(away_stats_df_list)

team_col = total_away_stats_df.pop("Team")
total_away_stats_df.insert(0, "Team", team_col)

total_away_stats_df

,Team,AB,R,H,RBI,BB,SO,PA,BA,OBP,...,aLI,WPA+,WPA-,cWPA,acLI,RE24,PO,A,Details,Date
[Hanley RamírezSS],Florida Marlins,5,1,3,4,0,0,6,.339,.391,...,1.01,0.245,-0.071,0.03%,0.26,1.7,4,2,"HR,2B,SF",7/27/2007
[Dan Uggla2B],Florida Marlins,6,1,1,0,0,0,6,.257,.331,...,0.93,0.039,-0.127,-0.01%,0.24,-1.0,1,3,2B,7/27/2007
[Miguel Cabrera3B],Florida Marlins,4,1,2,1,2,0,6,.332,.405,...,0.78,0.125,-0.051,0.01%,0.20,0.9,0,4,IW,7/27/2007
[Josh WillinghamLF],Florida Marlins,2,1,1,0,3,0,5,.263,.375,...,0.82,0.091,-0.046,0.01%,0.21,0.8,2,0,GDP,7/27/2007
[Mike Jacobs1B],Florida Marlins,5,1,2,2,0,1,5,.277,.328,...,1.35,0.250,-0.088,0.03%,0.35,0.7,8,1,2B,7/27/2007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
[Zach EflinP],Tampa Bay Rays,,,,,,,,,,...,,,,,,,0,0,,5/13/2024
[Kevin KellyP],Tampa Bay Rays,,,,,,,,,,...,,,,,,,1,0,,5/13/2024
[Garrett CleavingerP],Tampa Bay Rays,,,,,,,,,,...,,,,,,,0,0,,5/13/2024
[Jason AdamP],Tampa Bay Rays,,,,,,,,,,...,,,,,,,0,0,,5/13/2024


In [14]:
total_home_pitching_stats_df = pd.concat(home_pitching_stats_df_list)

team_col = total_home_pitching_stats_df.pop("Team")
total_home_pitching_stats_df.insert(0, "Team", team_col)

total_home_pitching_stats_df

,Team,IP,H,R,ER,BB,SO,HR,ERA,BF,...,Unk,GSc,IR,IS,WPA,aLI,cWPA,acLI,RE24,Date
Barry Zito,San Francisco Giants,4,8,6,6,4,3,1,5.28,23,...,0,21,,,-0.406,1.03,-0.01%,0.04,-3.9,7/27/2007
Kevin Correia,San Francisco Giants,1.1,2,1,1,1,2,1,4.21,7,...,0,,0,0,-0.138,0.59,-0.00%,0.02,-0.5,7/27/2007
"Vinnie Chulk, W (4-2)",San Francisco Giants,0.2,0,0,0,1,0,0,3.35,3,...,0,,1,0,0.059,1.80,0.00%,0.08,0.6,7/27/2007
Jonathan Sánchez,San Francisco Giants,1.2,2,1,1,1,1,0,4.45,8,...,0,,0,0,0.017,0.52,0.00%,0.02,-0.3,7/27/2007
Randy Messenger,San Francisco Giants,1.1,3,2,2,0,1,0,2.15,7,...,0,,1,0,0.027,1.04,0.00%,0.05,-1.1,7/27/2007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
New York Mets Totals,New York Mets,9,7,4,4,4,10,1,4.00,38,...,0,54,2,0,-0.052,0.57,-0.03%,0.49,0.6,5/11/2024
"Kutter Crawford, L (2-2)",Boston Red Sox,6,7,4,4,1,6,0,2.24,26,...,0,47,,,-0.066,0.96,-0.04%,0.90,-0.7,5/13/2024
Greg Weissert,Boston Red Sox,2,3,1,1,1,2,0,1.56,9,...,0,,0,0,-0.027,0.97,-0.01%,0.91,0.1,5/13/2024
Chase Anderson,Boston Red Sox,1,0,0,0,0,1,0,4.34,3,...,0,,0,0,0.020,0.26,0.01%,0.24,0.5,5/13/2024


In [15]:
total_away_pitching_stats_df = pd.concat(away_pitching_stats_df_list)

team_col = total_away_pitching_stats_df.pop("Team")
total_away_pitching_stats_df.insert(0, "Team", team_col)

total_away_pitching_stats_df

,Team,IP,H,R,ER,BB,SO,HR,ERA,BF,...,Unk,GSc,IR,IS,WPA,aLI,cWPA,acLI,RE24,Date
Rick van den Hurk,Florida Marlins,5,8,6,6,5,1,3,7.00,26,...,0,23,,,-0.386,0.93,-0.06%,0.24,-3.3,7/27/2007
"Renyel Pinto, BS (5), L (2-4)",Florida Marlins,0.2,3,4,4,1,1,1,4.24,6,...,0,,0,0,-0.425,1.46,-0.07%,0.37,-2.7,7/27/2007
Matt Lindstrom,Florida Marlins,0.1,3,2,1,0,1,0,3.98,4,...,0,,1,1,-0.141,0.42,-0.02%,0.11,-2.8,7/27/2007
Lee Gardner,Florida Marlins,1,0,0,0,0,0,0,2.85,3,...,0,,0,0,0.007,0.09,0.00%,0.02,0.5,7/27/2007
Taylor Tankersley,Florida Marlins,1,0,0,0,1,1,0,5.46,3,...,0,,0,0,0.003,0.09,0.00%,0.02,0.5,7/27/2007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Zach Eflin, W (3-4)",Tampa Bay Rays,5,6,3,3,0,2,1,3.91,20,...,0,45,,,0.029,1.03,0.01%,0.80,-0.3,5/13/2024
"Kevin Kelly, H (3)",Tampa Bay Rays,2,1,0,0,0,0,0,3.24,7,...,0,,0,0,0.202,1.71,0.09%,1.33,1.1,5/13/2024
"Garrett Cleavinger, H (4)",Tampa Bay Rays,1,1,0,0,0,3,0,1.72,4,...,0,,0,0,0.096,1.31,0.04%,1.01,0.5,5/13/2024
"Jason Adam, S (3)",Tampa Bay Rays,1,0,0,0,1,3,0,1.47,4,...,0,,0,0,0.100,2.46,0.05%,1.91,0.5,5/13/2024


In [16]:
total_game_logistics_df = pd.concat(game_logistics_df_list)
total_game_logistics_df

,Attendance,Game Duration,Venue,Date
0,"42,831",3:15,AT&T Park,7/27/2007
0,"43,001",2:55,AT&T Park,7/28/2007
0,"42,633",3:13,Busch Stadium III,9/5/2008
0,"42,814",2:24,Busch Stadium III,9/6/2008
0,"35,922",3:09,Angel Stadium of Anaheim,7/25/2009
...,...,...,...,...
0,"37,328",2:18,Busch Stadium III,4/6/2024
0,"39,519",2:27,Busch Stadium III,4/7/2024
0,"27,703",2:34,Oriole Park at Camden Yards,5/10/2024
0,"38,919",2:45,Citi Field,5/11/2024


In [17]:
index_list = total_box_score_df.index.tolist()
home_away_list = []


for i in range(len(index_list)):
    if (i % 2) == 0:
        home_away_list.append('Away')
    else:
        home_away_list.append('Home')


total_box_score_df = total_box_score_df.set_index('Date')
# total_box_score_df.insert(0, 'Team', index_list)
total_box_score_df.insert(0, 'Home_Away', home_away_list)
total_box_score_df

,Home_Away,,1,10,11,12,13,14,15,2,...,4,5,6,7,8,9,E,H,R,Team
Date,,,,,,,,,,,,,,,,,,,,,
"(7/27/2007,)",Away,via Sports Logos.netAbout logos,1,NaN,NaN,NaN,NaN,NaN,NaN,0,...,1,0,1,1,0,2,2,15,10,Florida Marlins
"(7/27/2007,)",Home,via Sports Logos.netAbout logos,2,NaN,NaN,NaN,NaN,NaN,NaN,0,...,0,4,6,0,0,X,0,14,12,San Francisco Giants
"(7/27/2007,)",Away,WP: Vinnie Chulk (4-2) • LP: Renyel Pinto (2-4),None,NaN,NaN,NaN,NaN,NaN,NaN,None,...,None,None,None,None,None,None,None,None,None,None
"(7/28/2007,)",Home,via Sports Logos.netAbout logos,0,NaN,NaN,NaN,NaN,NaN,NaN,0,...,0,0,0,0,0,0,2,6,3,Florida Marlins
"(7/28/2007,)",Away,via Sports Logos.netAbout logos,1,NaN,NaN,NaN,NaN,NaN,NaN,0,...,0,0,1,0,0,2,0,9,4,San Francisco Giants
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(5/11/2024,)",Away,via Sports Logos.netAbout logos,0,NaN,NaN,NaN,NaN,NaN,NaN,0,...,0,0,0,0,0,1,1,2,1,New York Mets
"(5/11/2024,)",Home,WP: Max Fried (3-1) • LP: Christian Scott (0-1),None,NaN,NaN,NaN,NaN,NaN,NaN,None,...,None,None,None,None,None,None,None,None,None,None
"(5/13/2024,)",Away,via Sports Logos.netAbout logos,3,NaN,NaN,NaN,NaN,NaN,NaN,0,...,1,0,0,0,1,0,0,10,5,Tampa Bay Rays


In [18]:
total_game_logistics_df.to_csv('total_game_logistics.csv')
total_away_stats_df.to_csv('total_away_stats.csv')
total_away_pitching_stats_df.to_csv('total_away_pitching_stats.csv')
total_home_stats_df.to_csv('total_home_stats.csv')
total_home_pitching_stats_df.to_csv('total_home_pitching_stats.csv')
total_box_score_df.to_csv('total_box_score_df.csv')
# pitching_line_df.to_csv('pitching_line_df.csv')
# hof_df.to_csv('mlb_hof.csv')
# mvp_df.to_csv('mlb_mvp.csv')
# all_league_df.to_csv('all_league_players.csv')
# allstar_df.to_csv('allstars.csv')
# agg_dpoy_seen_df.to_csv('dpoy.csv')

In [19]:
box_score_header_list

[['', 'Team', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'R', 'H', 'E']]